In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [3]:
train.isnull().sum()

ID                         0
var3                       0
var15                      0
imp_ent_var16_ult1         0
imp_op_var39_comer_ult1    0
                          ..
saldo_medio_var44_hace3    0
saldo_medio_var44_ult1     0
saldo_medio_var44_ult3     0
var38                      0
TARGET                     0
Length: 371, dtype: int64

In [20]:
train["TARGET"].nunique()

2

In [4]:
train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [5]:
train.shape

(76020, 371)

In [6]:
X = train.drop(['ID', 'TARGET'], axis=1)
y = train['TARGET']

In [7]:
#PCA on X
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
pca.fit(X)
X_pca = pca.transform(X)

In [11]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a pipeline with a StandardScaler and a XGBClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'xgb__max_depth': [2, 3, 4, 5, 6, None],
    'xgb__learning_rate': np.linspace(0.001, 0.1, 10),
    'xgb__n_estimators': [50, 100, 150]
}

# Perform RandomizedSearchCV with the pipeline
rgcv = RandomizedSearchCV(pipeline, param_distributions=param_grid, cv=5, scoring='neg_log_loss', random_state=42, n_iter=50, verbose=3)
rgcv.fit(X_train, y_train)

# Print the best score and best parameters
print("Best score: ", -rgcv.best_score_)  # Multiply by -1 because the score is the negative log loss
print("Best parameters: ", rgcv.best_params_)

# Predict the labels of the test set
y_pred = rgcv.predict(X_test)

# Compute the confusion matrix and F1 score
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and F1 score
print("Confusion Matrix: \n", cm)
print("F1 Score: ", f1)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.157 total time=   1.2s
[CV 2/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.157 total time=   1.2s
[CV 3/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.158 total time=   1.1s
[CV 4/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.156 total time=   1.1s
[CV 5/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.158 total time=   1.1s
[CV 1/5] END xgb__learning_rate=0.023000000000000003, xgb__max_depth=4, xgb__n_estimators=50;, score=-0.155 total time=   1.0s
[CV 2/5] END xgb__learning_rate=0.023000000000000003, xgb__max_depth=4, xgb__n_estimators=50;, score=-0.155 total time=   1.1s
[CV 3/5] END xgb__learning_rate=0.023000000000000003, xgb__max_depth=4, xgb__n_estimators=50;, score=-0.155 total time=

In [12]:
# save the best model
import joblib
joblib.dump(rgcv.best_estimator_, 'xgb_model.pkl')

['xgb_model.pkl']

In [13]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Create a pipeline with a StandardScaler and a XGBClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'xgb__max_depth': [2, 3, 4, 5, 6, None],
    'xgb__learning_rate': np.linspace(0.001, 0.1, 10),
    'xgb__n_estimators': [50, 100, 150]
}

# Perform RandomizedSearchCV with the pipeline
rgcv = RandomizedSearchCV(pipeline, param_distributions=param_grid, cv=5, scoring='neg_log_loss', random_state=42, n_iter=50, verbose=3)
rgcv.fit(X_train, y_train)

# Print the best score and best parameters
print("Best score: ", -rgcv.best_score_)  # Multiply by -1 because the score is the negative log loss
print("Best parameters: ", rgcv.best_params_)

# Predict the labels of the test set
y_pred = rgcv.predict(X_test)

# Compute the confusion matrix and F1 score
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and F1 score
print("Confusion Matrix: \n", cm)
print("F1 Score: ", f1)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.171 total time=   0.2s
[CV 2/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.172 total time=   0.2s
[CV 3/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.171 total time=   0.2s
[CV 4/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.171 total time=   0.1s
[CV 5/5] END xgb__learning_rate=0.012, xgb__max_depth=2, xgb__n_estimators=100;, score=-0.171 total time=   0.1s
[CV 1/5] END xgb__learning_rate=0.023000000000000003, xgb__max_depth=4, xgb__n_estimators=50;, score=-0.171 total time=   0.1s
[CV 2/5] END xgb__learning_rate=0.023000000000000003, xgb__max_depth=4, xgb__n_estimators=50;, score=-0.172 total time=   0.1s
[CV 3/5] END xgb__learning_rate=0.023000000000000003, xgb__max_depth=4, xgb__n_estimators=50;, score=-0.172 total time=

In [14]:
# save the best model
import joblib
joblib.dump(rgcv.best_estimator_, 'xgb_model_pca.pkl')

['xgb_model_pca.pkl']

In [15]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)

# Create a pipeline with a StandardScaler and a RandomForestClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=23))
])

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'rf__n_estimators': [10, 50, 100, 200],
    'rf__max_depth': [None, 5, 10, 15, 20],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}

# Perform RandomizedSearchCV with the pipeline
rgcv = RandomizedSearchCV(pipeline, param_distributions=param_grid, cv=5, scoring='neg_log_loss', random_state=23, n_iter=50, verbose=3)
rgcv.fit(X_train, y_train)

# Print the best score and best parameters
print("Best score: ", -rgcv.best_score_)  # Multiply by -1 because the score is the negative log loss
print("Best parameters: ", rgcv.best_params_)

# Predict the labels of the test set
y_pred = rgcv.predict(X_test)

# Compute the confusion matrix and F1 score
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and F1 score
print("Confusion Matrix: \n", cm)
print("F1 Score: ", f1)

# save the best model
import joblib
joblib.dump(rgcv.best_estimator_, 'rf_model_no_pca.pkl')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.148 total time=   0.7s
[CV 2/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.149 total time=   0.7s
[CV 3/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.150 total time=   0.7s
[CV 4/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.149 total time=   0.7s
[CV 5/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.146 total time=   0.6s
[CV 1/5] END rf__max_depth=20, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=50;, score=-0.138 total time=   4.2s
[CV 2/5] END rf__max_depth=20, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=50;, score=-0.138 total time=   4.2s
[CV 3

['rf_model_no_pca.pkl']

In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=23)

# Create a pipeline with a StandardScaler and a RandomForestClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=23))
])

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'rf__n_estimators': [10, 50, 100, 200],
    'rf__max_depth': [None, 5, 10, 15, 20],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}

# Perform RandomizedSearchCV with the pipeline
rgcv = RandomizedSearchCV(pipeline, param_distributions=param_grid, cv=5, scoring='neg_log_loss', random_state=23, n_iter=50, verbose=3)
rgcv.fit(X_train, y_train)

# Print the best score and best parameters
print("Best score: ", -rgcv.best_score_)  # Multiply by -1 because the score is the negative log loss
print("Best parameters: ", rgcv.best_params_)

# Predict the labels of the test set
y_pred = rgcv.predict(X_test)

# Compute the confusion matrix and F1 score
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and F1 score
print("Confusion Matrix: \n", cm)
print("F1 Score: ", f1)

# save the best model
import joblib
joblib.dump(rgcv.best_estimator_, 'rf_model_pca.pkl')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.167 total time=   0.2s
[CV 2/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.164 total time=   0.2s
[CV 3/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.165 total time=   0.2s
[CV 4/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.165 total time=   0.2s
[CV 5/5] END rf__max_depth=5, rf__min_samples_leaf=4, rf__min_samples_split=5, rf__n_estimators=10;, score=-0.167 total time=   0.1s
[CV 1/5] END rf__max_depth=20, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=50;, score=-0.167 total time=   0.7s
[CV 2/5] END rf__max_depth=20, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=50;, score=-0.164 total time=   0.7s
[CV 3

['rf_model_pca.pkl']

In [9]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a pipeline with a StandardScaler and a LGBMClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lgbm', LGBMClassifier(random_state=42))
])

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'lgbm__max_depth': [2, 3, 4, 5, 6, None],
    'lgbm__learning_rate': np.linspace(0.001, 0.999, 10),
    'lgbm__n_estimators': [50, 100, 150]
}

# Perform RandomizedSearchCV with the pipeline
rgcv = RandomizedSearchCV(pipeline, param_distributions=param_grid, cv=5, scoring='neg_log_loss', random_state=42, n_iter=50, verbose=1)
rgcv.fit(X_train, y_train)

# Print the best score and best parameters
print("Best score: ", -rgcv.best_score_)  # Multiply by -1 because the score is the negative log loss
print("Best parameters: ", rgcv.best_params_)

# Predict the labels of the test set
y_pred = rgcv.predict(X_test)

# Compute the confusion matrix and F1 score
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and F1 score
print("Confusion Matrix: \n", cm)
print("F1 Score: ", f1)

# save the best model
import joblib
joblib.dump(rgcv.best_estimator_, 'lgbm_model_no_pca.pkl')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Info] Number of positive: 1675, number of negative: 40896
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013135 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13577
[LightGBM] [Info] Number of data points in the train set: 42571, number of used features: 248
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.039346 -> initscore=-3.195219
[LightGBM] [Info] Start training from score -3.195219
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 1674, number of negative: 40897
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing wa

['lgbm_model_no_pca.pkl']

In [10]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import joblib


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Create a pipeline with a StandardScaler and a LGBMClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lgbm', LGBMClassifier(random_state=42))
])

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'lgbm__max_depth': [2, 3, 4, 5, 6, None],
    'lgbm__learning_rate': np.linspace(0.001, 0.999, 10),
    'lgbm__n_estimators': [50, 100, 150]
}

# Perform RandomizedSearchCV with the pipeline
rgcv = RandomizedSearchCV(pipeline, param_distributions=param_grid, cv=5, scoring='neg_log_loss', random_state=42, n_iter=50, verbose=3)
rgcv.fit(X_train, y_train)

# Print the best score and best parameters
print("Best score: ", -rgcv.best_score_)  # Multiply by -1 because the score is the negative log loss
print("Best parameters: ", rgcv.best_params_)

# Predict the labels of the test set
y_pred = rgcv.predict(X_test)

# Compute the confusion matrix and F1 score
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and F1 score
print("Confusion Matrix: \n", cm)
print("F1 Score: ", f1)

# save the best model
joblib.dump(rgcv.best_estimator_, 'lgbm_model_pca.pkl')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Info] Number of positive: 1675, number of negative: 40896
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002472 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 42571, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.039346 -> initscore=-3.195219
[LightGBM] [Info] Start training from score -3.195219
[CV 1/5] END lgbm__learning_rate=0.11188888888888888, lgbm__max_depth=2, lgbm__n_estimators=100;, score=-0.156 total time=   0.2s
[LightGBM] [Info] Number of positive: 1674, number of negative: 40897
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000623 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] T

['lgbm_model_pca.pkl']

In [17]:
#prepare test data
test = test.drop(['ID'], axis=1)
test_pca = pca.transform(test)

In [18]:
#load models
xgb_model = joblib.load('xgb_model_no_pca.pkl')
xgb_model_pca = joblib.load('xgb_model_pca.pkl')
rf_model = joblib.load('rf_model_no_pca.pkl')
rf_model_pca = joblib.load('rf_model_pca.pkl')
lgbm_model = joblib.load('lgbm_model_no_pca.pkl')
lgbm_model_pca = joblib.load('lgbm_model_pca.pkl')

In [19]:
#submission files
submission['TARGET'] = xgb_model.predict(test)
submission.to_csv('xgb_model.csv', index=False)

submission['TARGET'] = xgb_model_pca.predict(test_pca)
submission.to_csv('xgb_model_pca.csv', index=False)

submission['TARGET'] = rf_model.predict(test)
submission.to_csv('rf_model.csv', index=False)

submission['TARGET'] = rf_model_pca.predict(test_pca)
submission.to_csv('rf_model_pca.csv', index=False)

submission['TARGET'] = lgbm_model.predict(test)
submission.to_csv('lgbm_model.csv', index=False)

submission['TARGET'] = lgbm_model_pca.predict(test_pca)
submission.to_csv('lgbm_model_pca.csv', index=False)

In [ ]:
#